In [115]:
import matplotlib.pyplot as plt
from t4gpd.io.CirReader import CirReader
from t4gpd.pyvista.ToUnstructuredGrid import ToUnstructuredGrid
from shapely.geometry import Polygon
from geopandas import GeoDataFrame
import numpy as np
import pandas as pd
import geopandas
from math import *
from numpy.random import randint
from shapely.geometry import LineString,Point
from t4gpd.commons.GeomLib import GeomLib
from t4gpd.demos.GeoDataFrameDemos import GeoDataFrameDemos
from t4gpd.morph.geoProcesses.FootprintExtruder import FootprintExtruder
from t4gpd.morph.geoProcesses.STGeoProcess import STGeoProcess
from t4gpd.morph.STPointsDensifier2 import STPointsDensifier2
from t4gpd.pyvista.ToUnstructuredGrid import ToUnstructuredGrid
import random

In [116]:
#gdf = GeoDataFrameDemos.districtRoyaleInNantesBuildings()
gdf = CirReader("C:/Users/zcui/Desktop/programming/Re_ Rnion bilan GLO dans le SIG/scene_masque.cir").run()

In [117]:
def module(vector):
    return np.sqrt(np.dot(np.array(vector),np.array(vector)))

In [118]:
def cmp(a,b,c=1e-10):
    return abs(a-b)<c

In [119]:
def OnSurface(self,point):
    ang = 0
    p = np.array(point)
    coord = self.exterior.coords
    for i in range(len(coord)-1):
        vec1 = np.array(coord[i]) - p
        vec2 = np.array(coord[i+1])-p
        m1 = module(vec1)
        m2 = module(vec2)
        if cmp(m1*m2,0):
            return True
        else:
            cost = np.dot(vec1,vec2)/(m1*m2)
        ang = ang +np.arccos(cost)
    return cmp(ang,2*np.pi)

In [120]:
def ConvertCIR(gdf):
    basemap_x=[]
    basemap_y=[]
    basemap_z=[]
    sidewall_x=[]
    sidewall_y=[]
    sidewall_z=[]
    for i in range(len(gdf)):
        f = gdf.geometry[i]
        fig = Polygon(f)
        x, y, z=zip(*fig.exterior.coords)
        if np.std(z)==0:
            basemap_x.append(x)
            basemap_y.append(y)
            basemap_z.append(z)
        else:
            sidewall_x.append(x)
            sidewall_y.append(y)
            sidewall_z.append(z)
    
    elevation = []
    for i in range(len(basemap_z)):
        elevation.append(np.mean(basemap_z[i]))
    df = GeoDataFrame({'HAUTEUR':elevation},crs='epsg:2154')
    df['gid'] = df.index+1
    
    lable = []
    for i in range(len(df)):
        if df.HAUTEUR[i]==0:
            lable.append("Ground")
        else:
            lable.append("Building")
    df['Lable'] = lable
    
    geometry =[]
    for i in range(len(df)):
        p = Polygon(list(zip(basemap_x[i],basemap_y[i],basemap_z[i])))
        p2 = Polygon(list(zip(basemap_x[i],basemap_y[i])))
        geometry.append(p)
    df['geometry'] = geometry

    _difference = df.geometry[0]#create ground with holes
    for i in range(1,len(df)):
        _difference = _difference.difference(df.geometry[i])
    df.at[0,'geometry']=_difference
    
    floor = []
    for i in range(len(df)):
        n=0
        if df.Lable[i] == 'Ground':
            floor.append(0)
        else:
            for j in range(len(sidewall_x)):
                q = Polygon(list(zip(sidewall_x[j],sidewall_y[j])))
                if df.geometry[i].contains(q):
                    n = n+1
            nmb = len(basemap_x[i])-1
            floor.append(n/nmb)
    df['floor'] = floor
    return df

In [121]:
def RandomDir(n):#number of samples
    vector = []
    for i in range(n):
        polar = random.random()*pi
        azimuth = random.random()*pi
        x = sin(polar)*cos(azimuth)
        y = sin(polar)*sin(azimuth)
        z = cos(polar)
        vector.append(np.array([x,y,z]))
    return vector

In [122]:
def RayPoint(point,n,norm,length):
    ray_line = []
    vec = RandomDir(n)
    for i in range(len(vec)):
        vec1 = vec[i]
        x2 = norm[0]*vec1[1]+norm[1]*vec1[0]
        y2 = norm[1]*vec1[1]-norm[0]*vec1[0]
        z2 = vec1[2]
        p2 = Point(point.x+x2*length,point.y+y2*length,point.z+z2*length)
        line = LineString([point,p2])
        ray_line.append(line)
    line_geo = GeoDataFrame({'geometry':ray_line},crs='epsg:2154')
    return line_geo

In [123]:
def SurfaceNormal(self): #input is the buildingsIn3d.geometry[i],than choose the polygon of one facet
    coord = self.exterior.coords
    facet_p0=np.array(coord[0])
    facet_p1=np.array(coord[1])
    facet_p2=np.array(coord[2])
    vec1 = facet_p1 - facet_p0
    vec2 = facet_p2 - facet_p1
    a = vec1[1]*vec2[2]-vec1[2]*vec2[1]
    b = vec1[2]*vec2[0]-vec1[0]*vec2[2]
    c = vec1[0]*vec2[1]-vec1[1]*vec2[0]
    normal = np.array([a,b,c])
    mod = module(normal)
    n  = normal/mod
    return n

In [124]:
df = ConvertCIR(gdf)

In [125]:
df_buildings = GeoDataFrame.copy(df)#Here I create a copy value of df for puting sensors
df_buildings.index -= 1
df_buildings.drop(df_buildings.index[df_buildings['Lable'] == 'Ground'], inplace = True)
sensors = STPointsDensifier2(df_buildings, curvAbsc=[0.25,0.5,0.75], pathidFieldname=None).run()
sensors.floor = sensors.floor.astype(int)

In [126]:
from t4gpd.commons.GeomLib import GeomLib\

rows = []
for _,row in sensors.iterrows():
    for nfloor in range(row.floor):
        _row = row.copy()
        _row_z = nfloor*3.0+1.5
        _row.HAUTEUR = _row_z
        _row.floor = nfloor
        _row.geometry = GeomLib.forceZCoordinateToZ0(_row.geometry,_row_z)
        rows.append(_row)
sensors = GeoDataFrame(rows,crs=sensors.crs)
sensors.reset_index(inplace=True,drop=True)

In [127]:
op = FootprintExtruder(df_buildings, 'HAUTEUR', forceZCoordToZero=True)
buildingsIn3d = STGeoProcess(op, df_buildings).run()
scene1 = ToUnstructuredGrid([buildingsIn3d,sensors]).run()
scene1.plot(point_size=10.0, render_points_as_spheres=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [128]:
#create face_id for every facets
sur_gid = []
sur_geo = []
fid = []
for j in range(len(buildingsIn3d)):
    surface = buildingsIn3d.geometry[j]
    f_id=1
    for i in range(2,len(surface)): #only create the side wall geodataframe here
        sur_gid.append(buildingsIn3d.gid[j])
        sur_geo.append(surface[i])
        fid.append(f_id)
        f_id = f_id + 1
surf = GeoDataFrame({'gid':sur_gid},crs='epsg:2154')
surf['geometry'] = sur_geo
surf['fid'] = fid

In [129]:
#save surface normal vector
n = []
p_fid = []
for i in range(len(sensors)):
    #point2 = np.array([sensors2.geometry[i].x,sensors2.geometry[i].y,sensors2.geometry[i].z])
    point2 = sensors.geometry[i]
    for j in range(len(surf)):
        if point2.intersects(surf.geometry[j]):
            normal = SurfaceNormal(surf.geometry[j])
            p_fid.append(surf.fid[j])
            n.append(normal)
sensors['normalvector'] = n
sensors['fid'] = p_fid
sensors

,HAUTEUR,gid,Lable,geometry,floor,node_id,motion_dir,normalvector,fid
0,1.5,2,Building,POINT Z (70.000 55.000 1.500),0,0#0#0,0.0#1.0,"[1.0, 0.0, 0.0]",1
1,4.5,2,Building,POINT Z (70.000 55.000 4.500),1,0#0#0,0.0#1.0,"[1.0, 0.0, 0.0]",1
2,7.5,2,Building,POINT Z (70.000 55.000 7.500),2,0#0#0,0.0#1.0,"[1.0, 0.0, 0.0]",1
3,10.5,2,Building,POINT Z (70.000 55.000 10.500),3,0#0#0,0.0#1.0,"[1.0, 0.0, 0.0]",1
4,13.5,2,Building,POINT Z (70.000 55.000 13.500),4,0#0#0,0.0#1.0,"[1.0, 0.0, 0.0]",1
...,...,...,...,...,...,...,...,...,...
1075,10.5,10,Building,POINT Z (40.000 40.000 10.500),3,8#0#11,-1.0#0.0,"[0.0, 1.0, -0.0]",4
1076,13.5,10,Building,POINT Z (40.000 40.000 13.500),4,8#0#11,-1.0#0.0,"[0.0, 1.0, -0.0]",4
1077,16.5,10,Building,POINT Z (40.000 40.000 16.500),5,8#0#11,-1.0#0.0,"[0.0, 1.0, -0.0]",4
1078,19.5,10,Building,POINT Z (40.000 40.000 19.500),6,8#0#11,-1.0#0.0,"[0.0, 1.0, -0.0]",4


In [143]:
#sensors_2 = GeoDataFrame.copy(sensors[sensors.iloc[:,1]==2])
sensors_2 = GeoDataFrame.copy(sensors.loc[(sensors["gid"]==2) & (sensors["fid"]==2)])
sensors_2.reset_index(inplace=True,drop=True)

In [131]:
p = sensors_2.geometry[19]
normal = sensors_2.normalvector[19]
ray = RayPoint(p,1000,normal,20)

In [132]:
scene2 = ToUnstructuredGrid([buildingsIn3d,sensors_2,ray]).run()
scene2.plot(point_size=10.0, render_points_as_spheres=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [133]:
#problem: I have to test every point of the surface since maybe some parts of the surface are facing, some are not
def FaceSurface(sensor,surface):
    n1 = SurfaceNormal(surface)
    n2 = sensor.normalvector
    coords = surface.exterior.coords
    point = np.array(sensor.geometry)
    for j in range(1,len(coords)):
        vec = np.array(coords[j])-point
        con1 = vec.dot(n1)#/(module(vec)*module(n2))
        con2 = vec.dot(n2)
        if con2>0 and con1<0:
            return True
        else:
            continue
    return False

In [134]:
def intersection(ray,sensor,surface):
    n = SurfaceNormal(surface)
    coords = surface.exterior.coords
    point = np.array(sensor.geometry)
    vec = np.array(coords[1])-point
    con1 = ray.dot(n)
    con2 = np.dot(vec,n)
    ratio = con2/con1
    if con1>0:
        return False
    else:
        in_point = point + ratio*ray
        if OnSurface(surface,Point(in_point)):
            return ratio
        else:
            return False  

In [33]:
#select the facets facing the sensor
gid2=[]
fid2=[]
geo=[]
p2 = sensors_2.iloc[19]
for i in range(len(surf)):
    if FaceSurface(p2,surf.geometry[i])==True:
        gid2.append(surf.gid[i])
        fid2.append(surf.fid[i])
        geo.append(surf.geometry[i])
fac_surf = GeoDataFrame({'gid':gid2},crs='epsg:2154')
fac_surf['geometry'] = geo
fac_surf['fid'] = fid2

In [39]:
fac_surf['nray'] = 0
for i in range(len(ray)):
    ratio = []
    P1 = Point(ray.geometry[i].coords[0])
    P2 = Point(ray.geometry[i].coords[1])
    ray_vec = np.array([P2.x,P2.y,P2.z]) - np.array([P1.x,P1.y,P1.z])
    for j in range(len(fac_surf)):
        value = intersection(ray_vec,p2,fac_surf.geometry[j])
        if value==False:
            ratio.append(100)
        else:
            n=n+1 #in order to verify the number of intersection number
            ratio.append(value)
    fac_surf['ratio'] = ratio
    mymin = min(fac_surf.ratio)
    for z in range(len(fac_surf)):
        if fac_surf.ratio[z]==mymin and fac_surf.ratio[z]!=100:
            fac_surf.loc[z,'nray'] = fac_surf.loc[z,'nray']+1
fac_surf

,gid,geometry,fid,nray,ratio
0,5,"POLYGON Z ((75.000 90.000 0.000, 75.000 70.000...",3,157,100
1,7,"POLYGON Z ((60.000 80.000 0.000, 60.000 100.00...",1,45,100
2,7,"POLYGON Z ((40.000 80.000 0.000, 60.000 80.000...",4,106,100
3,9,"POLYGON Z ((20.000 75.000 0.000, 20.000 95.000...",1,22,100
4,9,"POLYGON Z ((0.000 75.000 0.000, 20.000 75.000 ...",4,4,100


In [145]:
for s in range(len(sensors_2)):
    #find the surface facing the sensor
    p = sensors_2.iloc[s]
    normal = sensors_2.normalvector[s]
    ray = RayPoint(p.geometry,1000,normal,20)
    gid2 = []
    fid2 = []
    geo = []
    for i in range(len(surf)):
        if FaceSurface(p,surf.geometry[i])==True:
            gid2.append(surf.gid[i])
            fid2.append(surf.fid[i])
            geo.append(surf.geometry[i])
    fac_surf = GeoDataFrame({'gid':gid2},crs='epsg:2154')
    fac_surf['geometry'] = geo
    fac_surf['fid'] = fid2
    
    #calculate the ratio between sensor and the facing surfaces
    fac_surf['nray'] = 0
    for a in range(len(ray)):
        ratio = []
        P1 = Point(ray.geometry[a].coords[0])
        P2 = Point(ray.geometry[a].coords[1])
        ray_vec = np.array([P2.x,P2.y,P2.z]) - np.array([P1.x,P1.y,P1.z])
        for j in range(len(fac_surf)):
            value = intersection(ray_vec,p2,fac_surf.geometry[j])
            if value==False:
                ratio.append(100)
            else:
                ratio.append(value)
        fac_surf['ratio'] = ratio
        mymin = min(fac_surf.ratio)
        for z in range(len(fac_surf)):
            if fac_surf.ratio[z]==mymin and fac_surf.ratio[z]!=100:
                fac_surf.loc[z,'nray'] = fac_surf.loc[z,'nray']+1
    print(fac_surf)

   gid                                           geometry  fid  nray  \
0    5  POLYGON Z ((75.000 90.000 0.000, 75.000 70.000...    3   152   
1    7  POLYGON Z ((60.000 80.000 0.000, 60.000 100.00...    1    35   
2    7  POLYGON Z ((40.000 80.000 0.000, 60.000 80.000...    4   100   
3    9  POLYGON Z ((20.000 75.000 0.000, 20.000 95.000...    1    15   
4    9  POLYGON Z ((0.000 75.000 0.000, 20.000 75.000 ...    4     2   

        ratio  
0  100.000000  
1  100.000000  
2    1.625647  
3  100.000000  
4  100.000000  
   gid                                           geometry  fid  nray  ratio
0    5  POLYGON Z ((75.000 90.000 0.000, 75.000 70.000...    3   163    100
1    7  POLYGON Z ((60.000 80.000 0.000, 60.000 100.00...    1    37    100
2    7  POLYGON Z ((40.000 80.000 0.000, 60.000 80.000...    4    97    100
3    9  POLYGON Z ((20.000 75.000 0.000, 20.000 95.000...    1    22    100
4    9  POLYGON Z ((0.000 75.000 0.000, 20.000 75.000 ...    4     2    100
   gid         

C:\Users\zcui\AppData\Local\Temp\ipykernel_20436\1592034024.py:14: RuntimeWarning: invalid value encountered in arccos
  ang = ang +np.arccos(cost)


   gid                                           geometry  fid  nray  ratio
0    5  POLYGON Z ((75.000 90.000 0.000, 75.000 70.000...    3   156    100
1    7  POLYGON Z ((60.000 80.000 0.000, 60.000 100.00...    1    36    100
2    7  POLYGON Z ((40.000 80.000 0.000, 60.000 80.000...    4   105    100
3    9  POLYGON Z ((20.000 75.000 0.000, 20.000 95.000...    1    26    100
4    9  POLYGON Z ((0.000 75.000 0.000, 20.000 75.000 ...    4     6    100
   gid                                           geometry  fid  nray  ratio
0    5  POLYGON Z ((75.000 90.000 0.000, 75.000 70.000...    3   161    100
1    7  POLYGON Z ((40.000 80.000 0.000, 60.000 80.000...    4    96    100
2    9  POLYGON Z ((20.000 75.000 0.000, 20.000 95.000...    1    23    100
3    9  POLYGON Z ((0.000 75.000 0.000, 20.000 75.000 ...    4     2    100
   gid                                           geometry  fid  nray  \
0    5  POLYGON Z ((75.000 90.000 0.000, 75.000 70.000...    3   164   
1    7  POLYGON Z ((

KeyboardInterrupt: 